In [19]:
import numpy as np
import pandas as pd
import yfinance as yf


In [ ]:
def brownianModel(TICKER, START_DATE, END_DATE, PRED_END_DATE):
    """ 
    Brownian model for stocks
    START_DATE: from which (past) date should we begin considering the Monte Carlo.
    END_DATE: from which (past) date are we ending the considering of its data (presumably as late as possible, given data is available)
    PRED_END_DATE: the (future) date which the client is willing to predict until.  
    """
    # main variables
    stock_name = TICKER
    start_date = START_DATE
    end_date = END_DATE
    pred_end_date = PRED_END_DATE
    scen_size = 10000

    # -----------------------------
    # Download and prepare data
    # -----------------------------

    prices = yf.download(tickers=stock_name, start=start_date, end=pred_end_date)

    # ---- FIX 1: Robust price column selection ----
    if isinstance(prices.columns, pd.MultiIndex):
        if ('Adj Close', stock_name) in prices.columns:
            prices = prices[('Adj Close', stock_name)]
        else:
            prices = prices[('Close', stock_name)]
    else:
        if 'Adj Close' in prices.columns:
            prices = prices['Adj Close']
        else:
            prices = prices['Close']

    
    # Generate business days (weekdays only)
    future_dates = pd.bdate_range(start=pd.to_datetime(end_date) + pd.Timedelta(days=1),
                    end=pd.to_datetime(pred_end_date))

   
    train_set = prices.loc[:end_date]
    # Create DataFrame with a 'Date' column
    dframe = pd.DataFrame({'Prediction Date': future_dates})
    
    daily_returns = ((train_set / train_set.shift(1)) - 1)[1:]



    So = train_set.iloc[-1]
    dt = 1  # day

    n_of_wkdays = pd.date_range(
        start=pd.to_datetime(end_date) + pd.Timedelta('1 days'),
        end=pd.to_datetime(pred_end_date)
    ).to_series().map(lambda x: 1 if x.isoweekday() in range(1, 6) else 0).sum()

    T = n_of_wkdays
    N = int(T / dt)
    t = np.arange(1, N + 1)

    mu = np.mean(daily_returns)
    sigma = np.std(daily_returns)

    b = {str(scen): np.random.normal(0, 1, N) for scen in range(1, scen_size + 1)}
    W = {str(scen): b[str(scen)].cumsum() for scen in range(1, scen_size + 1)}

    drift = (mu - 0.5 * sigma ** 2) * t
    diffusion = {str(scen): sigma * W[str(scen)] for scen in range(1, scen_size + 1)}

    S = np.array([So * np.exp(drift + diffusion[str(scen)]) for scen in range(1, scen_size + 1)])
    S = np.hstack((np.array([[So] for _ in range(scen_size)]), S))


    S_max = [S[:, i].max() for i in range(0, N)]
    S_min = [S[:, i].min() for i in range(0, N)]
    S_pred = 0.5 * np.array(S_max) + 0.5 * np.array(S_min)

    # Standard Monte Carlo estimator: expected price (mean across simulations)
    #S_pred = np.median(S, axis=0)[1:]



    # ---- FIX 2: Correct final_df construction (Series-safe) ----
    # Align prediction length with available real prices
    min_len = min(len(dframe['Prediction Date']), len(S_pred))

    final_df = pd.DataFrame({
        'pred': S_pred[:min_len]
    }, index=dframe['Prediction Date'])

    #mse = np.mean((final_df['pred'] - final_df['real']) ** 2)
    # Convert to string row by row as "Date: Price"

    result = '\n'.join(f"{date.date()}: {price}" for date, price in zip(final_df.index, final_df['pred']))
    return result


/var/folders/k3/b9g3nh5972v8265mx4qbp1tm0000gn/T/ipykernel_75146/39812135.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers=stock_name, start=start_date, end=pred_end_date)
[*********************100%***********************]  1 of 1 completed

<class 'str'>
